<a href="https://colab.research.google.com/github/apmoore1/target-extraction/blob/master/tutorials/Load_and_Explore_Target_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!pip uninstall target-extraction
!pip install git+git://github.com/apmoore1/target-extraction.git@master#egg=target-extraction

Uninstalling target-extraction-0.0.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/target_extraction-0.0.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/target_extraction/*
Proceed (y/n)? y
  Successfully uninstalled target-extraction-0.0.1
  Cloning git://github.com/apmoore1/target-extraction.git (to revision master) to /tmp/pip-install-c98jy2pi/target-extraction
  Running command git clone -q git://github.com/apmoore1/target-extraction.git /tmp/pip-install-c98jy2pi/target-extraction
  Stored in directory: /tmp/pip-ephem-wheel-cache-4hv_epdc/wheels/49/38/dd/3a8462a96b96857513b9f7f42036f80db8e77a32291f62e80e
Successfully built target-extraction


In [0]:
from target_extraction.dataset_parsers import semeval_2014, semeval_2016

# Loads and explore datasets
In this notebook we will show you how to load two of the popular SemEval datasets into the TargetTextCollection object format. Next we will show you the advantage of this easy to use format through serveral built in functions such as:

1. Dataset Statistics -- counts of sentences and targets.
2. Built in tokenization and POS tagging.
3. Import and export to JSON format which is useful if you want to use the data in an [AllenNLP model](https://allennlp.org/)
4. Setting the task of Target Extraction into a Sequence Labelling task

## Loading datasets
First we are going to show you how to load the following popular datasets:
1. [SemEval 2014 task 4 Laptop domain (laptop)](http://alt.qcri.org/semeval2014/task4/). Of which the training data can be found [here](http://metashare.ilsp.gr:8080/repository/browse/semeval-2014-absa-train-data-v20-annotation-guidelines/683b709298b811e3a0e2842b2b6a04d7c7a19307f18a4940beef6a6143f937f0/) and the test data [here](http://metashare.ilsp.gr:8080/repository/browse/semeval-2014-absa-test-data-gold-annotations/b98d11cec18211e38229842b2b6a04d77591d40acd7542b7af823a54fb03a155/).
2. [SemEval 2014 task 4 Restaurant domain (restaurant_14)](http://alt.qcri.org/semeval2014/task4/). Of which the training and the test data can be found at the same place as the laptop dataset.
3. [SemEval 2016 task 5 Restaurant domain (restaurant_16)](http://alt.qcri.org/semeval2016/task5/). Of which the training data can be found [here](http://metashare.ilsp.gr:8080/repository/browse/semeval-2016-absa-restaurant-reviews-english-train-data-subtask-1/cd28e738562f11e59e2c842b2b6a04d703f9dae461bb4816a5d4320019407d23/) and the test data [here](http://metashare.ilsp.gr:8080/repository/browse/semeval-2016-absa-restaurant-reviews-english-test-data-gold-subtask-1/42bd97c6d17511e59dbe842b2b6a04d721d1933085814d9daed8fbcbe54c0615/).

Assuming you have downloaded these files we will now upload them to this notebook, this will require 6 upload requests which will happen when you run the next cell; 1. The training, 2. test data for the laptop domain, 3. the training data, 4. test data for the restaurant_14, 5. the training data, and 6. test data for the restaurant_16

In [3]:
from pathlib import Path
from google.colab import files
uploaded = {}
for i in range(6):
  temp_uploaded = files.upload()
  uploaded = {**uploaded, **temp_uploaded}
semeval_fps = [Path(key).resolve() for key in uploaded.keys()]
semeval_fps = {semeval_path.name: semeval_path for semeval_path in semeval_fps}
del uploaded
# Paths to the files uploaded
for file_name, fp in semeval_fps.items():
  print(f'SemEval file name: {file_name}. File Path {fp}')

Saving Laptop_Train_v2.xml to Laptop_Train_v2 (2).xml


Saving Restaurants_Train_v2.xml to Restaurants_Train_v2 (2).xml


Saving Laptops_Test_Gold.xml to Laptops_Test_Gold (2).xml


Saving Restaurants_Test_Gold.xml to Restaurants_Test_Gold (2).xml


Saving EN_REST_SB1_TEST.xml.gold to EN_REST_SB1_TEST.xml (2).gold


Saving ABSA16_Restaurants_Train_SB1_v2.xml to ABSA16_Restaurants_Train_SB1_v2 (2).xml
SemEval file name: Laptop_Train_v2.xml. File Path /content/Laptop_Train_v2.xml
SemEval file name: Restaurants_Train_v2.xml. File Path /content/Restaurants_Train_v2.xml
SemEval file name: Laptops_Test_Gold.xml. File Path /content/Laptops_Test_Gold.xml
SemEval file name: Restaurants_Test_Gold.xml. File Path /content/Restaurants_Test_Gold.xml
SemEval file name: EN_REST_SB1_TEST.xml.gold. File Path /content/EN_REST_SB1_TEST.xml.gold
SemEval file name: ABSA16_Restaurants_Train_SB1_v2.xml. File Path /content/ABSA16_Restaurants_Train_SB1_v2.xml


We will now load the data into TargetTextCollection format:

NOTE: The `conflict` argument states whether or not to include targets or categories that contain the `conflict` sentiment. In this case we **do** want the `conflict` sentiment targets or categories thus the value is `False`

In [0]:
laptop_train = semeval_2014(semeval_fps['Laptop_Train_v2.xml'], conflict=False)
laptop_test = semeval_2014(semeval_fps['Laptops_Test_Gold.xml'], conflict=False)
rest_14_train = semeval_2014(semeval_fps['Restaurants_Train_v2.xml'], conflict=False)
rest_14_test = semeval_2014(semeval_fps['Restaurants_Test_Gold.xml'], conflict=False)
rest_16_train = semeval_2016(semeval_fps['ABSA16_Restaurants_Train_SB1_v2.xml'], conflict=False)
rest_16_test = semeval_2016(semeval_fps['EN_REST_SB1_TEST.xml.gold'], conflict=False)

## Dataset Statistics

Now that we have loaded these datasets we can perform some dataset statistics on them, such as the number of sentences, number of sentences that contain at least one target in it, and the number of  targets in the dataset.

In [5]:
from collections import defaultdict
import pandas as pd

name_dataset = [('Laptop Train', laptop_train), ('Laptop Test', laptop_test),
                ('Restaurant 14 Train', rest_14_train), 
                ('Restaurant 14_Test', rest_14_test),
                ('Restaurant 16 Train', rest_16_train), 
                ('Restaurant 16_Test', rest_16_test)]
dataset_stats = defaultdict(lambda: {})

for name, dataset in name_dataset:
  num_sentences = len(dataset)
  num_targ_sents = len(dataset.samples_with_targets())
  num_targs = sum([count for count in dataset.target_count().values()])
  
  dataset_stats[name]['Number Sentences'] = num_sentences
  dataset_stats[name]['Number Target Sentences'] = num_targ_sents
  dataset_stats[name]['Number Targets'] = num_targs
pd.DataFrame(dataset_stats).T  

,Number Sentences,Number Target Sentences,Number Targets
Laptop Train,3045,1488,2358
Laptop Test,800,422,654
Restaurant 14 Train,3041,2021,3693
Restaurant 14_Test,800,606,1134
Restaurant 16 Train,2000,1708,2507
Restaurant 16_Test,676,587,859


These statistics match those of the original SemEval task 4 subtask 1 2014 Laptop/Restaurant dataset [paper](https://www.aclweb.org/anthology/S14-2004), and original SemEval task 5 subtask 1 2016 Restaurant dataset [paper](https://www.aclweb.org/anthology/S16-1002).

However the Restaurant 2016 dataset is reporting the number of (targets, categories) which means that for the target extraction task this is not ideal as we can have duplicate targets with the same spans in the same sentence as each one of those duplicate targets can have a different category. An example of this is shown below:

In [6]:
sample = rest_16_train['1086415:2']
sample

TargetText({'text': '$6 and there is much tasty food, all of it fresh and continually refilled.', 'text_id': '1086415:2', 'targets': ['food', 'food', 'food'], 'spans': [Span(start=27, end=31), Span(start=27, end=31), Span(start=27, end=31)], 'target_sentiments': ['positive', 'positive', 'positive'], 'categories': ['FOOD#STYLE_OPTIONS', 'FOOD#QUALITY', 'FOOD#PRICES'], 'category_sentiments': None})

As we can see in the example above the target food with the eaxct same span is used three times as each time it has a different category associated to it. To convert the dataset into one that only allows one target with one span use the `one_sample_per_span` function to create a new TargetTextCollection dataset (NOTE: This is only required for the Restaurant 2016 dataset as the previous datasets did not link the target and categories together):



In [7]:
rest_target_16_train = rest_16_train.one_sample_per_span(remove_empty=True)
rest_target_16_test = rest_16_test.one_sample_per_span(remove_empty=True)

sample = rest_target_16_train['1086415:2']
sample

TargetText({'text': '$6 and there is much tasty food, all of it fresh and continually refilled.', 'text_id': '1086415:2', 'targets': ['food'], 'spans': [Span(start=27, end=31)], 'target_sentiments': None, 'categories': None, 'category_sentiments': None})

As we can see above the TargetText instance now only has `food` once but we have had to remove all of the category information and sentiment. However this is perfect if we want to just perform the Target Extraction task. 

NOTE: The `remove_empty=True` argument is required as some of the (target, category) pairs contain targets that are None as in the target does not exist but the category does therefore setting this argument to `True` removes these None targets

In [8]:
name_dataset = [('Laptop Train', laptop_train), ('Laptop Test', laptop_test),
                ('Restaurant 14 Train', rest_14_train), 
                ('Restaurant 14_Test', rest_14_test),
                ('Restaurant 16 Train', rest_target_16_train), 
                ('Restaurant 16_Test', rest_target_16_test)]
dataset_stats = defaultdict(lambda: {})

for name, dataset in name_dataset:
  num_sentences = len(dataset)
  num_targ_sents = len(dataset.samples_with_targets())
  num_targs = sum([count for count in dataset.samples_with_targets().target_count().values()])
  
  dataset_stats[name]['Number Sentences'] = num_sentences
  dataset_stats[name]['Number Target Sentences'] = num_targ_sents
  dataset_stats[name]['Number Targets'] = num_targs
pd.DataFrame(dataset_stats).T  

,Number Sentences,Number Target Sentences,Number Targets
Laptop Train,3045,1488,2358
Laptop Test,800,422,654
Restaurant 14 Train,3041,2021,3693
Restaurant 14_Test,800,606,1134
Restaurant 16 Train,2000,1235,1745
Restaurant 16_Test,676,421,613


Now that we have re-run the statistics of the dataset we can see that the Restaurant dataset has changed with regards to the Number of targets and target sentences. These statistics are now more in line with previous work ([link](https://www.aclweb.org/anthology/D17-1310) [link](https://www.aclweb.org/anthology/N19-1242)). However there are some reasons why they are not identical only for the Restaurant 2016 dataset, these are artifacts in the original annotation that we found which we share below:

### First annotation mistake
This is the case where the annotation stats NULL for the target but give a Span value:

``` xml
<sentence id="en_SchoonerOrLater_477965850:10">
  <text>In short, Schooner or Later couldn't have cared less about our being at their establishment.</text>
  <Opinions>
    <Opinion target="NULL" category="SERVICE#GENERAL" polarity="negative" from="10" to="27"/>
  </Opinions>
</sentence>
```
In these cases we change target from `NULL` to `Schooner or Later` and more generally to the text that the span refers to.

These mistakes add more targets to the dataset of which in the Restaurant train and test dataset there were 3 (1 of which linked to an exisitng target as in that target now existed twice with difference categories) and 1 mistakes respectively.

### Second annotation mistake
Annotation stats NULL for the target but the Span value is given but makes no sense:

``` xml
<sentence id="en_MercedesRestaurant_478010600:1">
  <text>– !</text>
  <Opinions>
    <Opinion target="service" category="SERVICE#GENERAL" polarity="positive" from="39" to="46"/>
    <Opinion target="NULL" category="FOOD#QUALITY" polarity="positive" from="2" to="21"/>
  </Opinions>
</sentence>
```
As we can see that there is a target which is `NULL` but the spans refers to the following text `rcedes restaurant i` which just makes no sense. Therefore in these cases we leave the target as it does not exist and change the spans to `from = 0` and `to = 0`. In these case they do not affect the overall statistics it just highlights mistakes that have been made. In the Restaurant train and test dataset there were 1 and 4 mistakes respectively.

## Tokenization

The `target_extraction` has a few tokenization options avaliable:
1. [Spacy](https://spacy.io/) -- Rule based tokenizer. 
2. [Stanford](https://stanfordnlp.github.io/stanfordnlp/), related [paper](https://www.aclweb.org/anthology/K18-2016) -- Neural Network based tokenizer.
3. Whitespace.

Both the Spacy and Stanford tokenizers support multiple languages.

To use any of the tokenizers we have to call the tokenizers method to get a related tokenizer as shown below:

In [9]:
from target_extraction import tokenizers

example_sentence = "I've had a great day today. The tokenizer's work well."

# Get different spacy tokenizers
# English is the default language
spacy_tok_en = tokenizers.spacy_tokenizer()
# German
spacy_tok_de = tokenizers.spacy_tokenizer(lang='de')


# Get different Stanford tokenizers
# English again is the default language and the default treebank is EWT
stanford_tok_en = tokenizers.stanford()
# Dutch but with a treebank that is not default
stanford_tok_nl = tokenizers.stanford(lang='nl', treebank='lassysmall')


# Whitespace
whitespace_tok = tokenizers.whitespace()

print(f'Spacy: {spacy_tok_en(example_sentence)}')
print(f'Stanford: {stanford_tok_en(example_sentence)}')
print(f'Whitespace {whitespace_tok(example_sentence)}')


Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/nl_lassysmall_models/nl_lassysmall_tokenizer.pt', 'lang': 'nl', 'shorthand': 'nl_lassysmall', 'mode': 'predict'}
Done loading processors!
---
Spacy: ['I', "'ve", 'had', 'a', 'great', 'day', 'today', '.', 'The', 'tokenizer', "'s", 'work', 'well', '.']
Stanford: ['I', "'ve", 'had', 'a', 'great', 'day', 'today', '.', 'The', 'tokenizer', "'s", 'work', 'well', '.']
Whitespace ["I've", 'had', 'a', 'great', 'day', 'today.', 'The', "tokenizer's", 'work', 'well.']


As we can see the Spacy and the Stanford tokenizers are more similar than the whitespacy with regards to handling full stops and hyphons.

We can also use these tokenizers to tokenizse the whole of our datasets easily as shown below (in the example below we use the Stanford tokenizer):

In [10]:
# Tokenize each dataset
for name, dataset in name_dataset:
  print(name)
  dataset.tokenize(stanford_tok_en)

sample = rest_target_16_train['1086415:2']
print(f"Text: {sample['text']}\nTokens: {sample['tokenized_text']}")

Laptop Train
Laptop Test
Restaurant 14 Train
Restaurant 14_Test
Restaurant 16 Train
Restaurant 16_Test
Text: $6 and there is much tasty food, all of it fresh and continually refilled.
Tokens: ['$', '6', 'and', 'there', 'is', 'much', 'tasty', 'food', ',', 'all', 'of', 'it', 'fresh', 'and', 'continually', 'refilled', '.']


We sample within the TargetTextCollection now has a new property (key in the internal disctionary) `tokenized_text` which contains the tokenized text for that sample.

## POS Tagging
The `target_extraction` has a couple of POS tagging options avaliable:
1. [Spacy](https://spacy.io/) -- Neural Network based. 
2. [Stanford](https://stanfordnlp.github.io/stanfordnlp/), related [paper](https://www.aclweb.org/anthology/K18-2016) -- Neural Network based.

Both the Spacy and Stanford POS taggers support multiple languages. They also both support two different types of tag sets:
1. [Universal POS tags (coarse)](https://universaldependencies.org/u/pos/) -- These are the same tags accross languages.
2. Language Specific (fine) -- These are language independent tags and are usuall more fine grained than the Universal, an example of this tag set would be the [Penn Treebank](https://www.clips.uantwerpen.be/pages/mbsp-tags).

To use any of the POS taggers we have to call the POS tagger's method to get the related POS tagger as shown below:

In [11]:
from target_extraction import pos_taggers

example_sentence = "I've had a great day today. The tokenizer's work well."

# Get different spacy tokenizers
# English is the default language with the small model and here we have chosen 
# the universal tag set
spacy_pos_en = pos_taggers.spacy_tagger(fine=False)
# German with the small model and language independent tag set
spacy_pos_de = pos_taggers.spacy_tagger(spacy_model_name='de_core_news_sm', 
                                        fine=True)


# Get different Stanford tokenizers
# English again is the default language and the default treebank is EWT, 
# using language independent tag set.
stanford_pos_en_fine = pos_taggers.stanford(fine=True)
# Dutch but with a treebank that is not default and language independent tag set
stanford_pos_nl = pos_taggers.stanford(lang='nl', treebank='lassysmall', fine=True)
# English again is the default language and the default treebank is EWT, 
# using the universal tag set
stanford_pos_en_coarse = pos_taggers.stanford(fine=False)

print(f'Stanford Language Independent: {stanford_pos_en_fine(example_sentence)}')
print(f'Stanford Universal: {stanford_pos_en_coarse(example_sentence)}')
print(f'Spacy: {spacy_pos_en(example_sentence)}')


Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/nl_lassysmall_models/nl_lassysmall_tokenizer.pt', 'lang': 'nl', 'shorthand': 'nl_lassysmall', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/nl_lassysmall_models/nl_lassysmall_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/nl_lassysmall_models/nl_lassysmall.pretrain.pt', 'lang': 'nl', 'shorthand': 'nl_lassysmall', 'mode': 'predict'}
Done loading processors!
---
Use device: 

We can easily see the difference between the universal and independent tag sets.

Like the tokenizers we can easily POS tag the whole dataset as shown below: 

**NOTE** even the POS taggers do the whole pipeline of tokenization and POS tagging it does expect that the dataset has already been tokenized with the same tokenizer i.e. Stanford with the same language and treebank.

In [12]:
# POS tag each dataset
for name, dataset in name_dataset:
  print(name)
  dataset.pos_text(stanford_pos_en_coarse)

sample = rest_target_16_train['1086415:2']
print(f"Text: {sample['text']}\nTokens: {sample['tokenized_text']}\nPOS tags: {sample['pos_tags']}")

Laptop Train
Laptop Test
Restaurant 14 Train
Restaurant 14_Test
Restaurant 16 Train
Restaurant 16_Test
Text: $6 and there is much tasty food, all of it fresh and continually refilled.
Tokens: ['$', '6', 'and', 'there', 'is', 'much', 'tasty', 'food', ',', 'all', 'of', 'it', 'fresh', 'and', 'continually', 'refilled', '.']
POS tags: ['SYM', 'NUM', 'CCONJ', 'PRON', 'VERB', 'ADV', 'ADJ', 'NOUN', 'PUNCT', 'DET', 'ADP', 'PRON', 'ADJ', 'CCONJ', 'ADV', 'VERB', 'PUNCT']


## To and From JSON

Once loaded into the TargetTextCollection format it is easy to export to JSON format and import into a TargetTextCollection from JSON as shown below:

In [13]:
from target_extraction.data_types import TargetTextCollection

example_json = laptop_train.to_json()[:200]
print(f'Sample of the JSON: {example_json}')
empty_collection = TargetTextCollection.from_json(laptop_train.to_json())
empty_collection == laptop_train

Sample of the JSON: {"text": "I charge it at night and skip taking the cord with me because of the good battery life.", "text_id": "2339", "targets": ["cord", "battery life"], "spans": [[41, 45], [74, 86]], "target_senti


True

As we can see above we can export to a JSON string and import from a JSON string. Below we show similar methods but instead of exporting and importing from Strings we do this from Files or more specifically File Paths:

In [14]:
from pathlib import Path
import tempfile

with tempfile.NamedTemporaryFile(mode='w+') as temp_file:
  temp_fp = Path(temp_file.name)
  laptop_train.to_json_file(temp_fp)
  empty_collection = None
  empty_collection = TargetTextCollection.load_json(temp_fp)
  print(empty_collection == laptop_train)

True


And as expected the TargetTextCollection loaded from the JSON file is the same as the one that exported it into JSON.

## Creating the sequence labelling task for Target Extraction and the True Upper limit
The Target Extraction (TE) task within Aspect Based Sentiment Analysis (ABSA) is normally formulated as a Sequence Labelling task ([paper link](https://www.aclweb.org/anthology/P18-2094)), however unlike other sequence labelling problems that have pre-tokenized text with gold sequence labels like [NER](https://www.aclweb.org/anthology/P16-1101) TE is evaluated based on Eaxct Span Matching not Sequence Labels.

This means that if we do treat the task as a sequence labelling task we can have tokenization errors and we show this below and thus show (we believe) for the first time the true upper limit of TE when treating it as a sequence labelling task due to tokenization error.

In [18]:
# Just to let you know for the Future to perform sequence_labels it requires
# you to tokenize the TargetTextCollection first
for name, dataset in name_dataset:
  print(name)
  dataset.sequence_labels()
sample = rest_target_16_train['1086415:2']
print(sample['sequence_labels'])
print(sample['tokenized_text'])
print(sample['targets'])
print(sample['spans'])

Laptop Train
Laptop Test
Restaurant 14 Train
Restaurant 14_Test
Restaurant 16 Train
Restaurant 16_Test
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['$', '6', 'and', 'there', 'is', 'much', 'tasty', 'food', ',', 'all', 'of', 'it', 'fresh', 'and', 'continually', 'refilled', '.']
['food']
[Span(start=27, end=31)]


Above we have used the Gold Spans to create the sequence labels based on the tokenization and we have printed the sequence labels, tokens, and the Gold Spans and in this case the Spans align perfectly with the tokens. However the assumption I think in the field is that tokenization does not cause any errors but this is not True as not all tokens perfectly align with the Gold Spans an example of this is shown below. 

In [32]:
measures = laptop_train.exact_match_score('sequence_labels')
recall, precision, f1, errors_analysis = measures
print(f'Recall: {recall:.3f}\nPrecision: {precision:.3f}\nF1: {f1:.3f}\n')

false_positive_mistakes = errors_analysis['FP']
false_negative_mistakes = errors_analysis['FN']
print(f'Example of mistake return {false_positive_mistakes[0]}')
fp_sample_mistake_id = false_positive_mistakes[0][0]
fp_sample_mistake_span = false_positive_mistakes[0][1]

fp_sample = laptop_train[fp_sample_mistake_id]
incorrect_target = fp_sample['text'][fp_sample_mistake_span.start:fp_sample_mistake_span.end]
print(f'FP Mistake using `Golden` sequence labels:\nText: {fp_sample["text"]}'
      f'\nTokenised Text: {fp_sample["tokenized_text"]}\n'
      f'Gold Sequence Labels: {fp_sample["sequence_labels"]}\n'
      f'Gold Spans: {fp_sample["spans"]}\nGold Targets: {fp_sample["targets"]}\n'
      f'The Span that is the mistake: {fp_sample_mistake_span}\n'
      f'The Target the incorrect span picked out: {incorrect_target}')

Recall: 0.997
Precision: 0.997
F1: 0.997

Example of mistake return ('2534', Span(start=57, end=69))
FP Mistake using `Golden` sequence labels:
Text: But the machine is awesome and iLife is great and I love Snow Leopard X.
Tokenised Text: ['But', 'the', 'machine', 'is', 'awesome', 'and', 'i', 'Life', 'is', 'great', 'and', 'I', 'love', 'Snow', 'Leopard', 'X.']
Gold Sequence Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O']
Gold Spans: [Span(start=31, end=36), Span(start=57, end=71)]
Gold Targets: ['iLife', 'Snow Leopard X']
The Span that is the mistake: Span(start=57, end=69)
The Target the incorrect span picked out: Snow Leopard


We can see why it did not pick out `Snow Leopard X` as a Multi Word Target because the tokenization did not Seperate `X` with the `.` thus `X.` was not labelled and even if it was it would be incorrect as `Snow Leopard X.` is not equal to `Snow Leopard X`

Below we show what the true F1 scores are for the different datasets with the three different tokenization methods.

In [0]:
name_tokenizer = [('Stanford', stanford_tok_en), ('Spacy', spacy_tok_en),
                  ('Whitespace', whitespace_tok)]
dataset_tokenizer = defaultdict(lambda: {})
for dataset_name, dataset in name_dataset:
  for tokenizer_name, tokenizer in name_tokenizer:
    dataset.tokenize(tokenizer)
    dataset.sequence_labels()
    measures = dataset.exact_match_score('sequence_labels')
    f1 = measures[2]
    dataset_tokenizer[dataset_name][tokenizer_name] = f1
pd.DataFrame(dataset_tokenizer).T.round(3)

In [36]:
pd.DataFrame(dataset_tokenizer).T.round(3)

,Spacy,Stanford,Whitespace
Laptop Train,0.995,0.997,0.749
Laptop Test,0.993,0.992,0.715
Restaurant 14 Train,0.998,0.999,0.777
Restaurant 14_Test,0.999,1.000,0.727
Restaurant 16 Train,0.998,0.999,0.793
Restaurant 16_Test,0.995,1.000,0.757


Above the F1 scores for the different tokenizers of which these sequences would be treated as the Gold Standard Sequences for the machine learning model show very different results between an actual tokenizer and Whitespace. The difference between Stanford and Spacy is marginal